# Import modules

In [ ]:
%load_ext autoreload
# # 1 means: only reload modules marked with "%aimport"
# # 2 means: reload all modules
%autoreload 2

In [ ]:
#thread management
from Queue import Queue
from threading import Thread,Lock
from collections import deque

#Picamera
import picamera as pc
from picamera.array import PiRGBArray

#image manipulation
import numpy as np
import pandas as pd
import cv2 #opencv
import matplotlib.pyplot as plt
%matplotlib inline

#general modules
import time,datetime
import pickle
import sys, warnings
import copy
from PIL import Image

#for fitting train-direction model
from scipy.optimize import curve_fit
import math




# Initialize Picamera and Capture Frames from Video Stream

In [ ]:
class Video_Camera(Thread):
    def __init__(self,fps,width,height,vflip,hflip,mins):
        self.fps=fps
        self.width=width
        self.height=height
        self.vflip=vflip
        self.hflip=hflip
        self.mins=mins
        #Deque storage data structures
        #Set max length of X minutes to prevent memory errors
        self.input_deque=deque(maxlen=fps*mins*60) 
        #start the thread, deamon and kill switch
        super(Video_Camera, self).__init__()
        self.daemon = True
        self.kill_all_threads= False
        print self, 'created'
        #Initialize camera and video stream
        self.initialize_camera()
        self.initialize_video_stream()
        print 'Camera and video stream initialized'
        
    def initialize_camera(self):
        self.camera = pc.PiCamera(
            resolution=(self.width,self.height), 
            framerate=int(self.fps))
        #Set camera properties
        self.camera.vflip = self.vflip
        self.camera.hflip = self.hflip
        self.apply_camera_day_settings()
        
    def initialize_video_stream(self):
        self.rawCapture = pc.array.PiRGBArray(self.camera, size=self.camera.resolution) 
        self.stream = self.camera.capture_continuous(self.rawCapture,
             format="bgr", 
             use_video_port=True)
    
    def apply_camera_day_settings(self):
        self.camera.exposure_mode = 'auto'
        self.camera.contrast=0
        self.camera.brightness=50
        self.camera.exposure_compensation=0
        #assign a custom flag
        self.camera.operating_mode='day'

    def apply_camera_night_settings(self):
        self.camera.exposure_mode = 'auto'
        self.camera.contrast=20
        self.camera.brightness=90
        self.camera.exposure_compensation=6
        #assign a custom flag
        self.camera.operating_mode='night'
        
    def run(self):
        #This method is run when the command start() is given to the thread
        print 'Video stream is now being captured'
        for f in self.stream:
            #add frame with timestamp to input queue
            self.input_deque.append({
                'time':time.time(),
                'frame_raw':f.array})
            
            #remove the frame from the stream 
            self.rawCapture.truncate(0)
            
            if self.kill_all_threads==True:
                print self, 'terminated'
                break
                
                
#Initialize Video_Camera Thread
video_camera = Video_Camera(fps=5,
                            width=384,
                            height=216,
                            vflip=True,
                            hflip=False,
                            mins=1)
#Begin capturing raw video and store data in the input_queue
video_camera.start()

# Save the input stream to file for future use

In [ ]:
#Save input queue to file
save_stream_to_file=False
if save_stream_to_file==True:
    print video_camera.input_deque[0]
    pickle.dump(video_camera.input_deque,open("persisted_input_queue_20160624_b","wb"))

In [ ]:
#Load input queue from file
load_stream_from_file=True
if load_stream_from_file==True:
    pickle_object=pickle.load( open("persisted_input_queue_20160624_b","rb"))
    video_camera.kill_all_threads=True
    time.sleep(1)
    video_camera.input_deque=pickle_object

# Create a background subtractor to help detect motion

In [ ]:
class Mask():
    def __init__(self,fps):
        self.fps=fps
        self.mask=None
        
    def make_mask(self,mask_type):
        if mask_type=='KNN':
            #K-nearest neigbours - based Background/Foreground Segmentation Algorithm.
            self.bgsKNN_history=int((1.0/self.fps)*40.0) #length of history in seconds
            self.bgsKNN_d2T=400 #distance to threshold
            self.bgsKNN_dS=False #detect shadows
            #create and return mask
            self.mask=self.make_KNN_mask(self.bgsKNN_history,self.bgsKNN_d2T,self.bgsKNN_dS)
        elif mask_type=='MOG2':
            #Gaussian Mixture-based Background/Foreground Segmentation Algorithm - Improved!
            self.detectShadows = True
            self.mask=self.make_MOG2_mask(self.detectShadows)
        else:
            print 'Error: Incorrect Mask Type Chosen!'
        return self.mask
    
    def make_KNN_mask(self,bgsKNN_history,bgsKNN_d2T,bgsKNN_dS):
        mask = cv2.createBackgroundSubtractorKNN(\
            history=bgsKNN_history,\
            dist2Threshold=bgsKNN_d2T,\
            detectShadows=bgsKNN_dS)
        return mask
        
    def make_MOG2_mask(self,detectShadows):
        mask = cv2.createBackgroundSubtractorMOG2(detectShadows = detectShadows)
        return mask

# Process the captured frames, dynamically Vary Camera Settings

In [ ]:
class Video_Sensor(Thread):
    def __init__(self,video_camera,mask_type):
        #Assign objects from the video camera class
        self.video_camera=video_camera
        self.camera=self.video_camera.camera
        self.input_deque=self.video_camera.input_deque
        self.fps=self.video_camera.fps
        self.mins=self.video_camera.mins
        #Deque storage data structures
        #Set max length of X minutes to prevent memory errors
        self.output_deque=deque(maxlen=self.fps*self.mins*60) 
        #start the thread, deamon and kill switch
        super(Video_Sensor, self).__init__()
        self.daemon = True
        self.kill_all_threads= False
        print self, 'created'
        #Assign mask objects and create mask
        self.mask_type=mask_type
        self.create_mask()
        
    def create_mask(self):
        #create mask_object for future generations of masks
        self.mask_object=Mask(fps=video_camera.fps)
        self.mask=self.mask_object.make_mask(self.mask_type)
        print '%s created, with mask_type: %s'%(self.mask_object, self.mask_type)
        
    def vary_camera_settings(self,frame_raw):
        # Dynamically Vary Camera Settings
        intensity_mean=frame_raw.ravel().mean() #8 bit camera
        #adjust camera properties dynamically if needed, then reset mask
        if ((intensity_mean < (255.0/8) ) & (self.camera.operating_mode=='day')):
            self.video_camera.apply_camera_night_settings()
            time.sleep(1)
            self.mask=self.mask_object.make_mask(self.mask_type)
            print 'Day Mode Activated - Camera'
        if ((intensity_mean < (255.0*(3/4)) ) & (self.camera.operating_mode=='night')):
            self.video_camera.apply_camera_day_settings()
            time.sleep(1)
            self.mask=self.mask_object.make_mask(self.mask_type)
            print 'Night Mode Activated - Camera'
        return intensity_mean,self.mask

    def apply_mask_and_decrease_noise(self,frame_raw):
        #apply the background subtraction mask
        frame_motion = self.mask.apply(frame_raw)
        #apply morphology mask to decrease noise
        frame_motion_output = cv2.morphologyEx(
            frame_motion,\
            cv2.MORPH_OPEN,\
            kernel=np.ones((2,2),np.uint8))
        return frame_motion_output
        
    def run(self):
        print 'Video stream is now being processed, and camera settings varied if needed'
        while self.kill_all_threads!=True:
            try:
                #obtain raw data, apply background mask and decrease noise
                data = self.input_deque.popleft()
                frame_raw=data['frame_raw']
                frame_motion=self.apply_mask_and_decrease_noise(frame_raw)

                # Dynamically Vary Camera Settings, update mask if needed
                intensity_mean,self.mask=\
                    self.vary_camera_settings(frame_raw)

                #save these frames as a dict to an output queue
                self.output_deque.append({\
                    'frame_raw':frame_raw,\
                    'frame_motion':frame_motion,
                    'time':data['time'],
                    'intensity_mean':intensity_mean})

            except IndexError:
                #index error occurs when the input_queue is empty
                time.sleep(1/self.fps)
                pass
        
#Initialize Video_Sensor Thread
video_sensor = Video_Sensor(video_camera=video_camera,
                            mask_type='KNN')
video_sensor.start()

# Create a history dataframe to persist data

In [ ]:
class History():
    def __init__(self,fps):
        self.len_history=int(fps*600)
        self.columns=['time','left_roi','middle_roi','right_roi',\
                 'motion_detected','train_detected','direction'] 
        
    def setup_history(self):
        #create pandas dataframe that contains  column information
        self.history = pd.DataFrame.from_records(\
            np.zeros((self.len_history,len(self.columns))),
            index=np.arange(self.len_history),
            columns=self.columns)
        print 'History dataframe created with columns:\n', self.columns
        return self.history

# Video Detector Worker - Includes video plotter, direction detector and frame sampler

In [ ]:
class Detector_Worker(Thread):
    def __init__(self,video_sensor,video_detector):
        self.video_sensor=video_sensor
        self.video_detector=video_detector
        self.history=self.video_detector.history
        self.frame_sampler_buffer=self.video_detector.frame_sampler_buffer
        self.all_rois=self.video_detector.all_rois
        self.intensity_threshold=self.video_detector.motion_threshold
        self.fps=self.video_sensor.fps
        self.motion_threshold=self.video_detector.motion_threshold
        self.time_threshold=self.video_detector.time_threshold
        #start the thread, deamon and kill switch
        super(Detector_Worker, self).__init__()
        self.daemon = True
        self.kill_all_threads= False
        print self, 'created'
    
    '''
    UTILITY FUNCTIONS (below)
    '''
    
    def run(self):
        if self.kill_all_threads!=True:
            time.sleep(0.5)

    def get_history_snapshot(self):
            #grab and process the raw data
            self.history_snapshot_0=(self.history.left_roi / 255.0).tolist()
            self.history_snapshot_1=(self.history.middle_roi / 255.0).tolist()
            self.history_snapshot_2=(self.history.right_roi /255.0).tolist()
            self.history_snapshot_motion_events=(self.history.motion_detected.tolist())
            self.history_snapshot_detection_events=(self.history.train_detected.tolist())
            self.history_snapshot_direction_events=(self.history.direction.tolist())

            #obtain the average values from the motion of the ROIs
            self.history_snapshot_mean=[]
            for i in range(0,len(self.history_snapshot_0)):
                self.history_snapshot_mean.append((\
                    self.history_snapshot_0[i]+self.history_snapshot_1[i]+self.history_snapshot_2[i])/3.0)

            #import time and convert relative to relative_time
            self.original_time_xaxis=self.history.time.tolist()
            #set relative time to the most recent time in the time series
            self.relative_time=max(self.original_time_xaxis)
            self.time_string=datetime.datetime.fromtimestamp(\
                self.relative_time).strftime('%Y:%m:%d:%H:%M:%S')
            #normalize the time_xaxis to relative_time (by copying the original)
            self.time_xaxis=list(self.original_time_xaxis) 
            self.time_xaxis[:] = [x - self.relative_time for x in self.time_xaxis] 

    def identify_frame_of_train_detection(self):
        time_of_train=None
        for i in range(0,len(self.history_snapshot_detection_events)):
            if self.history_snapshot_detection_events[i]==True:
                #check if the timestamp is the most recent time
                if  self.original_time_xaxis[i] > time_of_train : 
                    time_of_train=self.original_time_xaxis[i]
                    train_detected_flag=True
        #if there is no train_detection event found, return last frame in deque
        if time_of_train!=None:
            return time_of_train, train_detected_flag
        else:
            train_detected_flag=False
            return max(self.original_time_xaxis),train_detected_flag
    
    
    '''
    VIDEO PLOTTER (below)
    '''
    def video_plotter(self,total_time_in_plot):
        #refresh the video_detector_worker history snapshot
        self.get_history_snapshot()
        #determine time length of plot
        self.total_time_in_plot=total_time_in_plot
        plt.figure(figsize=(15,7.5))
        plt.ylim([0,1.1])
        plt.xlim([-1*self.total_time_in_plot/self.fps,0])
        plt.xlabel(('Time: relative to %s (seconds)')%(self.time_string),size=20)
        plt.ylabel('Relative Motion (1 is high)', size=20)
        #add series to the plot
        plt.plot(self.time_xaxis,self.history_snapshot_0,'m',\
                marker='o', linewidth=2,label="ROI: South")
        plt.plot(self.time_xaxis,self.history_snapshot_1,'g',\
                marker='o', linewidth=2,label="ROI: Middle")
        plt.plot(self.time_xaxis,self.history_snapshot_2,'b',\
                marker='o', linewidth=2,label="ROI: North")
        plt.plot(self.time_xaxis,self.history_snapshot_motion_events,color='0.75',\
                linewidth=2,label="Motion Detected")
        plt.plot(self.time_xaxis,self.history_snapshot_detection_events,'k',\
                linewidth=3, label="Train Detected")
        #draw threshold line
        plt.axhline(y=(self.motion_threshold/255.0), xmin=0, xmax=1, linewidth=1,\
                    color = 'r', label="Detection Threshold")
        #draw legend and title
        plt.legend(loc='upper left',prop={'size':12})
        plt.title("Video Detection \n Requires: %s motion, for %s seconds"
                  % (str(round((self.motion_threshold/255.0),2)),\
                     str(self.time_threshold/self.fps)),size=20)
        plt.show()
        plt.close()
        
    '''
    FRAME SAMPLER (below)
    '''
    def draw_rois(self,frame):
        for v in self.all_rois:
            cv2.rectangle(frame,
                (v[0]),(v[1]),
                (255,0,0),#color
                2)#thickness
        return frame
    
    def get_frame_by_timestamp_from_frame_sampler_buffer(self,timestamp):
        for i in range(0,len(self.frame_sampler_buffer)-1):
            frame_time=self.frame_sampler_buffer[i]['time']
            frame_time_next=self.frame_sampler_buffer[i+1]['time']
            #check if the timestamp is inbetween the two frames
            if (timestamp>=frame_time) & (timestamp<=frame_time_next):
                return self.frame_sampler_buffer[i]
        #if no frame is found, grab the most recent frame
        return self.frame_sampler_buffer[-1]
    
    def frame_sampler(self):
        #refresh the video_detector_worker history snapshot
        self.get_history_snapshot()
        try:
            #identify if recent frame has a picture of the train
            timestamp,train_detected_flag = self.identify_frame_of_train_detection()
            images=self.get_frame_by_timestamp_from_frame_sampler_buffer(timestamp)
            if train_detected_flag==True:
                print 'Analyzing train detection event at frame:',i
            else:
                print 'Analyzing most recent frame'

            #create plot
            plt.figure(figsize=(16,8))
            #adjust raw image to RGB and plot
            plt.subplot(2,2,1)
            frame_raw=images['frame_raw']
            plt.imshow(np.flipud(frame_raw[:, :, ::-1]))
            #draw on the ROI of interest and plot
            plt.subplot(2,2,2)
            frame_raw=self.draw_rois(images['frame_raw'].copy())
            plt.imshow(np.flipud(frame_raw[:, :, ::-1]))
            #plot masked iamged
            plt.subplot(2,2,3)
            frame_motion=images['frame_motion']
            plt.imshow(np.flipud(\
                        255-frame_motion),\
                       cmap='Greys',  interpolation='nearest')
            #draw on the ROI of interest and plot
            plt.subplot(2,2,4)
            frame_motion=self.draw_rois(images['frame_motion'])
            plt.imshow(np.flipud(\
                        255-frame_motion),\
                       cmap='Greys',  interpolation='nearest')
            plt.show()
            plt.close()
        except:
            print 'No images in frame sampler buffer'
      
    
    '''
    TRAIN DIRECTION (below)
    '''     
    def extract_data_in_time_range(self,time_before,time_after,event_time):
        #find the indices that correspond to the time range
        indices = [i for i, x in enumerate(self.original_time_xaxis)\
                   if ((x <= (event_time+time_after)) and (x >= (event_time-time_before))) ]

        #extract the data in a 5 element array from history that correspond to these indices
        data=[self.original_time_xaxis,
             self.history_snapshot_0,
             self.history_snapshot_1,
             self.history_snapshot_2,
             self.history_snapshot_detection_events]
        extracted_data=[]
        for array in data:
            extracted_data.append([ array[i] for i in indices])
        return extracted_data
    
    def calculate_time_of_arrival(self,extracted_data,j,event_time):
        t,xdata,ydata=self.import_data_from_roi(extracted_data,j,event_time)
        #attempt to fit the curve with sigmoid
        curve_fit_possible,popt,pcov=self.km_map_wth_sigmoid(xdata,ydata)
        #plot the information
        x_curve = np.linspace(xdata.min(),xdata.max(), len(xdata))
        #if the curve can be fit accurately, use the fit funciotn
        if curve_fit_possible == True:
            try:
                y_curve = self.curve_func(xdata, *popt)
                km= (-math.log(2*-popt[0]-popt[2])) / popt[1]
            except:
                #if there are problems with a log of a negative number
                print 'cannot take a log of a neg number'
                km=self.alternate_km_map(ydata,t,event_time)
                y_curve=ydata
        else:
            #get the max value from the ROI sensor over the time frame stored
            km=self.alternate_km_map(ydata,t,event_time)
            #set the model y_curve to exactly match the data
            y_curve=ydata
        return km,y_curve,x_curve,xdata,ydata,t
    
    def import_data_from_roi(self,extracted_data,i,event_time):
        t=extracted_data[0]
        roi_of_interest=extracted_data[i]
        xdata=np.array([t[i] - event_time for i in range(0,len(t))])
        ydata=[roi_of_interest[i] for i in range(0,len(roi_of_interest))]
        return t,xdata,ydata

    def plot_for_train_direction(self,t_adjusted,ydata,x_curve,y_curve,j):
        color=['mp','bs','g^']
        color_model=['m','b','g']
        label_raw=['ROI: South','ROI: Middle','ROI: North']
        label_model=['ROI: South - FIT','ROI: Middle - FIT','ROI: North - FIT']
        #plot raw data
        plt.plot(t_adjusted, ydata, color[j-1],label=label_raw[j-1])
        #plot modelled curve
        plt.plot(x_curve, y_curve, color_model[j-1],label=label_model[j-1])

    def curve_func(self,x, a, b,c):
        #Sigmoid function
        return -a/(c+ np.exp(b * -x))
    
    def decide_train_direction(self,time_of_arrival):
        if time_of_arrival[0] < time_of_arrival[-1]:
            train_direction='NORTH'
        else:
            train_direction='SOUTH'
        return train_direction
    
    def format_entire_plot_for_train_direction(self,d,t_adjusted,ydata,x_curve,y_curve,\
        time_span_before,time_span_after,train_direction,time_string,time_of_arrival):
        #plot the detector spike then show plot
        plt.plot(t_adjusted,d,'k',label='Train Detection Event')
        plt.legend(loc='upper left',prop={'size':12})
        plt.xlabel('Time: relative to now (seconds)',size=20)
        plt.ylabel('Relative Motion (1 is high)', size=20)
        plt.title(("Train Detected Going: %s \n Plot of: %s sec to %s sec after train detected \nAt: %s")\
                      % (str(train_direction),str(time_span_before), str(time_span_after),\
                      str(time_string)),size=20)
        plt.figtext(.35, -.1, 'ROI South, time of arrival:'+str(time_of_arrival[0])+\
                    '\nROI Middle, time of arrival:'+str(time_of_arrival[1])+\
                    '\nROI North, time of arrival:'+str(time_of_arrival[2]),size=20)
        
    def clean_extracted_data(self,extracted_data,event_time,intensity_threshold):
        if extracted_data != None:
            #make a copy of the extracted data
            extracted_data_original=list(extracted_data)
            #clean the extracted_data values in the ROIs
            #(prevent dropping back to zero, and normalize to 100)
            for i in range (1,4):
                #walk through all values
                detection_limit_passed = False
                for j in range(0,len(extracted_data[i])):
                    #normalize to 100
                    extracted_data[i][j]=((extracted_data[i][j]))
                    #prevent the values from dropping back to zero
                    if j!=0:
                        #check the values against the video intensity threshold
                        if extracted_data[i][j] > ((intensity_threshold)):
                            variance_allowed=0.05
                            #enforce that the value of the next item cannot drop by more than 10%
                            if extracted_data[i][j] < ((extracted_data[i][j-1] * (1-variance_allowed))):
                                previous_value=extracted_data[i][j-1]
                                extracted_data[i][j] = previous_value
                        #check the values against a hardset intensity threshold, which is larger variance, lower threshold
                        elif extracted_data[i][j] > (0.25):
                            variance_allowed=0.10
                            #enforce that the value of the next item cannot drop by more than 10%
                            if extracted_data[i][j] < ((extracted_data[i][j-1] * (1-variance_allowed))):
                                previous_value=extracted_data[i][j-1]
                                extracted_data[i][j] = previous_value
            #get the time array
            t=extracted_data[0]
            return extracted_data,t,extracted_data_original

    def km_map_wth_sigmoid(self,xdata,ydata):
        try:
            popt, pcov = curve_fit(self.curve_func, xdata, ydata)
            curve_fit_possible = True
        except:
            #print 'unable to fit the curve'
            curve_fit_possible = False
            popt=None
            pcov=None    
        return curve_fit_possible,popt,pcov    
    
    def alternate_km_map(self,ydata,t,event_time):
        #determine emperically where the ROI sensor hits 50% of the max value
        max_value = max(ydata)
        for i in range(0,len(ydata)):
            #if the value is above half of the max value
            if ydata[i] > max_value/2.0:
                km=t[i]-event_time
                return km
        #if the value never exceeds half of the max value
        #return the end of the time series
        km=t[-1]-event_time
        return km
    
    def train_direction(self,time_before,time_after):
        #refresh the video_detector_worker history snapshot
        self.get_history_snapshot()
        #extracted data is a 5 element array, consisting of
        #time, 3 ROIs, and boolean detection events
        #with data points corresponding to time before/after detection
        timestamp,train_detected_flag = self.identify_frame_of_train_detection()
        print timestamp,train_detected_flag 
        #only proceed if a train was detected in the history dataframe (returns most recent)
        if train_detected_flag==True:
            event_time=timestamp
            extracted_data=self.extract_data_in_time_range(time_before,time_after,event_time)
            extracted_data,t,extracted_data_original=\
                self.clean_extracted_data(extracted_data,event_time,self.intensity_threshold)
            #create a new figure for plotting and an array to save the event times from each ROI
            plt.figure(figsize=(20,8))
            time_of_arrival=[]
            #calculate the time of arrivel in each ROI 
            for j in range(1,4):
                km,y_curve,x_curve,xdata,ydata,t=self.calculate_time_of_arrival(
                                                    extracted_data,j,event_time)
                time_of_arrival.append(km)
                #Get the normalized time and exact time of arrival - only do this once
                if j==1:
                    t_adjusted = [t[i] - event_time for i in range(0,len(t))]
                    time_string=datetime.datetime.fromtimestamp(event_time).strftime('%Y-%m-%d_%H-%M-%S-%f')
                self.plot_for_train_direction(t_adjusted,ydata,x_curve,y_curve,j)
                #print 'ROI:',j-1,'time of arrival',km
            #decide if the train was going north or south
            train_direction=self.decide_train_direction(time_of_arrival)
            #format plot 
            d=extracted_data[4] #to plot the detection event
            self.format_entire_plot_for_train_direction(d,t_adjusted,ydata,x_curve,y_curve,\
                    time_before,time_after,train_direction,time_string,time_of_arrival)
            #show the figure then return the train direction
            plt.show()
            plt.close()
            return train_direction
        else:
            print 'No train detected'
            return None

# Create Video Detector, determine motion for aReas Of Interest (ROIs), if train passed, and train direction

In [ ]:
class Video_Detector(Thread):
    def __init__(self,video_camera,video_sensor,\
                 motion_threshold,time_threshold,cooldown_period):
        #Assign objects from the video camera and sensor classes
        self.video_camera=video_camera
        self.video_sensor=video_sensor
        self.input_deque=self.video_camera.input_deque
        self.output_deque=self.video_sensor.output_deque
        self.fps=self.video_camera.fps
        self.framenum_at_train_detection = 0
        
        #Create ROIs with hardcoded aReas Of Interest (ROIs)
        self.create_rois()
        
        #Assign motion, time and cooldown value 
        self.motion_threshold=motion_threshold
        self.time_threshold=time_threshold #in frames
        self.cooldown_period=cooldown_period
        
        #create circular buffers for train motion, detection events and sampled frames
        self.create_buffers()
        
        #Create the history dataframe
        history_object=History(self.fps)
        self.history=history_object.setup_history()
        
        #Create the detector worker for determining train direction, and sample plotting
        self.detector_worker=Detector_Worker(video_sensor=self.video_sensor,
                                        video_detector=self)
        
        #start the thread, deamon and kill switch
        super(Video_Detector, self).__init__()
        self.daemon = True
        self.kill_all_threads= False
        print self, 'created'
     
    
    def create_rois(self):
        #hardcode aReas Of Interest, ROI: ((x1, y1), (x2, y2))
        left_roi=((2,80),(50,135))
        center_roi=((145,90),(215,130))
        right_roi=((325,100),(380,130))
        self.all_rois=[left_roi,center_roi,right_roi]
        
    def create_buffers(self):
        #create buffers for detecting if train recently passed and populate with 0s
        self.train_detected_buffer=deque(maxlen=self.cooldown_period)
        self.train_direction_buffer=deque(maxlen=self.cooldown_period)
        #make the motion detected buffer shorter than cooldown_period
        #length of motion_detected_buffer determines how time of motion translates into detection
        self.motion_detected_buffer=deque(maxlen=self.time_threshold)
        #prefill these buffers to max length
        for i in range(0,self.cooldown_period):
            self.motion_detected_buffer.append(0)
            self.train_detected_buffer.append(0)
            self.train_direction_buffer.append(0)
        #create frame sampler buffer (do not prefill this buffer)
        self.frame_sampler_buffer=deque(maxlen=self.cooldown_period)
        
    def extract_roi_and_process(self,frame_motion):
        for v in self.all_rois:
            #extract ROI to process from numpy array 
            #using the x1,y1,x2,y2 coordinates (v)
            ROI_to_process= frame_motion[v[0][1]:v[1][1],v[0][0]:v[1][0]]
            #get average motion by summing pixel values
            #then dividing by total pixels
            average_motion_per_roi = ROI_to_process.ravel().sum()\
                // ROI_to_process.ravel().shape[0] 
            #wait for the background subtractor mask to load before adding motion data
            if self.framenum < (5 * self.fps):
                average_motion_per_roi=0
            #update roi_data array (that will be a future row in the History DF)
            self.roi_data.append(average_motion_per_roi)
        
    def check_motion_and_update_motion_buffer(self):
        #check if motion exceeds the defined threshold:
        average_motion=reduce(lambda x, y: x + y, self.roi_data[1:4])\
            / len(self.roi_data[1:4])
        if average_motion > self.motion_threshold:
            self.roi_data.append(True)
            self.motion_detected_buffer.append(1)
        else:
            self.roi_data.append(False)
            self.motion_detected_buffer.append(0)
            
    def save_frame_sampler_to_file(self,data_time,train_direction):
        pickle.dump(self.frame_sampler_buffer,open("output/persisted_frame_sampler"+\
                                            data_time+str(train_direction),"wb"))
        #pickle_object=pickle.load( open("persisted_input_queue_20160706_a","rb"))
        
    def check_temporal_threshold_and_update_detection_buffer(self,data):
        motion_detected_average=reduce(lambda x, y: x + y, self.motion_detected_buffer)\
                /len(self.motion_detected_buffer) 
        #make sure all frames in motion buffer detected motion
        if motion_detected_average >= 1:
            #make sure that there has only been 1 train detected in detection buffer
            #i.e. that a cooldown period has been passed so trains are flagged only once
            if 1 not in self.train_detected_buffer:
                #Temporarily update History as well as roi_data for train_detection script
                self.roi_data.append(True)
                self.roi_data.append('NORTH') #default to NORTH for now
                self.history.iloc[self.framenum % self.history.shape[0]] = self.roi_data
                #add the positive event to the train_detected buffer
                self.train_detected_buffer.append(1)
                #DETERMINE TRAIN DIRECTION
                train_direction=self.detector_worker.train_direction(time_before=10.0,
                                time_after=1.0)
                #update the last position of the roi_data, as was already set to True
                self.roi_data[-1]=train_direction
                #add the train direction to the buffer
                self.train_direction_buffer.append(train_direction)
                data_time=datetime.datetime.fromtimestamp(\
                    data['time']).strftime('%Y_%m_%d_%H_%M_%S')
                print 'Train detected going %s at %s'%(data_time,train_direction)
                #save the frame_sampler to file
                self.save_frame_sampler_to_file(data_time,train_direction)
                #for copying frame_sampler to non-updating version
                self.framenum_at_train_detection=self.framenum
            else:
                self.update_buffers_for_no_train()
        else:
            self.update_buffers_for_no_train()
    
    def update_buffers_for_no_train(self):
        #As a train was detected in a cooldown period, 
        #no additional detection or direction event is logged
        self.train_detected_buffer.append(0)
        self.train_direction_buffer.append(None)
        #append roi_data (row for History DF) for train detection event
        self.roi_data.append(False)
        #append roi_data (row for History DF) for train direction event
        self.roi_data.append(None)
        
    def run(self):
        print 'ROI data is now being extracted and persisted'
        #keep track of index for inserting data into history df
        self.framenum=0
        while self.kill_all_threads!=True:
            #grab the motion frame (255 is motion, 0 is no motion) 
            try:
                data=self.output_deque.popleft()
                frame_motion=data['frame_motion']

                #roi_data is an array for appending a row to the History DF 
                #time of image-capture is first column in History DF 
                self.roi_data=[data['time']]

                #extract ROI to process and get average motion per ROI
                self.extract_roi_and_process(frame_motion)

                #check if motion exceeds the defined threshold and update buffers,roi_data:
                self.check_motion_and_update_motion_buffer()

                #check if temporal threshold exceeded for detection (& cooldown not violated)
                self.check_temporal_threshold_and_update_detection_buffer(data)

                #update the history dataframe and adjust the frame number pointer
                self.history.iloc[self.framenum % self.history.shape[0]] = self.roi_data
                self.framenum+=1
                
                #copying frame_sampler to non-updating version
                if ((self.framenum_at_train_detection != 0) & (self.framenum==self.framenum_at_train_detection+25)):
                    self.frame_sampler_buffer_copy=copy.deepcopy(self.frame_sampler_buffer)

                #add frames and data to sampler
                self.frame_sampler_buffer.append({
                    'frame_raw':data['frame_raw'],
                    'frame_motion':data['frame_motion'],
                    'time':data['time'],
                    'roi_data':self.roi_data,
                    'intensity_mean':data['intensity_mean']})
                
            except IndexError:
                #index error occurs when the output_queue is empty
                time.sleep(1/self.fps)
                pass
        
        
video_detector=Video_Detector(video_camera=video_camera,
                              video_sensor=video_sensor,
                              motion_threshold=255/4.0, #8 bit camera
                              time_threshold=int(video_camera.fps*2), #in frames
                              cooldown_period=int(video_camera.fps*10))
video_detector.start()

### Print properties of system

In [ ]:
print 'mask_type:',video_sensor.mask_type
print 'bgsKNN_history:',video_sensor.mask_object.bgsKNN_history
print 'bgsKNN_d2T:',video_sensor.mask_object.bgsKNN_d2T
print 'bgsKNN_dS:',video_sensor.mask_object.bgsKNN_dS


In [ ]:
print video_detector.framenum
video_detector.history[200%(int(video_sensor.fps*600)):240%(int(video_sensor.fps*600))]

In [ ]:
video_detector.detector_worker.train_direction(time_before=10.0,
                                time_after=1.0)

In [ ]:
#Load input queue from file
load_stream_from_file=False
if load_stream_from_file==True:
    pickle_object=pickle.load( open("persisted_input_queue_20160624_b","rb"))
    video_camera.kill_all_threads=True
    time.sleep(1)
    video_camera.input_deque=pickle_object

In [ ]:
video_detector.detector_worker.frame_sampler()

In [ ]:
video_detector.detector_worker.video_plotter(total_time_in_plot=video_sensor.fps*60)

## Get Sampled Frames

# Shutdown camera

In [ ]:
for i in range(0,len(video_detector.frame_sampler_buffer_copy)):
    image_type='frame_motion'
    image=video_detector.frame_sampler_buffer_copy[i][image_type]
    filename="output_video/"+ image_type + str(i)+".jpeg"
    plt.figure(figsize=(4,4))
    if (image_type=="frame_raw"):
        plt.imshow(np.flipud(image[:, :, ::-1]))
    if (image_type=="frame_motion"):
        plt.imshow(np.flipud(\
            255-image),\
            cmap='Greys',  interpolation='nearest')
    plt.savefig(filename)
    plt.close()

In [ ]:
time.sleep(1)

In [ ]:
kill=False
if kill==True:
    kill_all_threads=True
    camera.stop_recording()
    camera.close()